<a href="https://colab.research.google.com/github/Buse-cetin/Topic_Modelling/blob/main/Bertopicc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
datafile = '/content/drive/MyDrive/unwanted_clean.csv'

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import pandas as pd
import numpy as np
query_df = pd.read_csv(datafile,error_bad_lines=False)
query_df = query_df.iloc[:8000]
query_df['tweet'] = query_df['tweet'].astype(str)
query_df['tweet'].head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


0    dakikadan fazladır taksi dksı görünen otobüsü ...
1    olimpiyat cadde üzeri pendik buraya araç park ...
2    parklarda kontrol bakımları aksatarak canlarla...
3    parklarda kontrol bakımları aksatarak canlarla...
4    bugün bir taksici terörüne tanık oldumtaksici ...
Name: tweet, dtype: object

In [4]:
import re
import string
from gensim import corpora, models, similarities 

Tweetlerin emojilerden ve sembollerden temizlenmesi

In [5]:
import re

def deEmoji(text):

  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emojiler
          u"\U0001F300-\U0001F5FF"  # ifadeler
          u"\U0001F680-\U0001F6FF"  # semboller
          u"\U0001F1E0-\U0001F1FF"  
                            "]+", flags=re.UNICODE)
  return str(emoji_pattern.sub('', text) )

Stopwords 

In [6]:
with open('/content/drive/MyDrive/stopwords-tr (1).txt', 'r') as f:
    myList = [line.strip() for line in f]

Tokenization

In [8]:
def word_tokenize(sentence):
      
    
    acronym_each_dot = r"(?:[a-zğçşöüı]\.){2,}"
    acronym_end_dot = r"\b[a-zğçşöüı]{2,3}\."
    suffixes = r"[a-zğçşöüı]{3,}' ?[a-zğçşöüı]{0,3}"
    numbers = r"\d+[.,:\d]+"
    any_word = r"[a-zğçşöüı]+"
    punctuations = r"[a-zğçşöüı]*[.,!?;:]"
    word_regex = "|".join([acronym_each_dot,
                           acronym_end_dot,
                           suffixes,
                           numbers,
                           any_word,
                           punctuations])
    
    sentence = re.compile("%s"%word_regex, re.I).findall(sentence)
    return sentence

Büyük-küçük harf dönüşümü

In [ ]:
def initial_clean(text):
     
     text = text.translate(str.maketrans('', '', string.punctuation))
     text = text.lower() 
     text = word_tokenize(text)
     return text

In [ ]:
def remove_stop_words(text):
     stop_words = myList
     new_stopwords = ["pkk","imamoğlu","akp","chp","terör","izmir","iki","ibb","ibbsikayet","istanbul","olimpiyat","belediye","blk","den","soylu","belediye"]
     stop_words.extend(new_stopwords)
     return [word for word in text if word not in stop_words]

İstenmeyen kelimelerin çıkartılması

In [ ]:
unwanted_data= [ "pkk","imamoğlu","akp","chp","terör","izmir","iki","ibb","ibbsikayet","istanbul","nin","belediye","blk","olimpiyat","soylu","belediye"] 
query_df = pd.DataFrame(query_df, columns = ["tweet"])
unwanted= query_df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in unwanted_data))

print(unwanted)

0       dakikadan fazladır taksi dksı görünen otobüsü ...
1       cadde üzeri pendik buraya araç park etmek yasa...
2       parklarda kontrol bakımları aksatarak canlarla...
3       parklarda kontrol bakımları aksatarak canlarla...
4       bugün bir taksici terörüne tanık oldumtaksici ...
                              ...                        
7995    bir mesai çıkışlarında yol çilesi olmasaydı iy...
7996                               manifestliyo anlamadim
7997    asgariücret zam yansımaları erken başladı topl...
7998    vesile yi kınıyor ekonomi yönetimleri başarısı...
7999    kes tatavayi ibo peynirin kilosu kiymanin kilo...
Name: tweet, Length: 8000, dtype: object


Yukarıdaki tüm fonksiyonları tek bir işleve toplar

In [ ]:
def apply_all(text):
     
     return remove_stop_words(initial_clean(deEmoji(text)))

Token İşlemi

In [ ]:
import time   
t1 = time.time()   
query_df['tokenized_texts'] = query_df['tweet'].apply(apply_all) 
t2 = time.time()  
print("Time to clean and tokenize", len(query_df), "texts:", (t2-t1)/60, "min") 

Time to clean and tokenize 8000 texts: 7.756832003593445 min


In [ ]:
query_df.head(6)

,tweet,tokenized_texts
0,dakikadan fazladır taksi dksı görünen otobüsü ...,"[dakikadan, fazladır, taksi, dksı, görünen, ot..."
1,olimpiyat cadde üzeri pendik buraya araç park ...,"[cadde, üzeri, pendik, buraya, araç, park, etm..."
2,parklarda kontrol bakımları aksatarak canlarla...,"[parklarda, kontrol, bakımları, aksatarak, can..."
3,parklarda kontrol bakımları aksatarak canlarla...,"[parklarda, kontrol, bakımları, aksatarak, can..."
4,bugün bir taksici terörüne tanık oldumtaksici ...,"[bugün, taksici, terörüne, tanık, oldumtaksici..."
5,akşam otobüse binemedikkendinizle gurur duyun ...,"[akşam, otobüse, binemedikkendinizle, gurur, d..."


BerTopic

In [ ]:
!pip install bertopic[visualization] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 65.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [ ]:
docs = list(query_df.loc[:, "tweet"].values)

In [ ]:
docs[:5]

['dakikadan fazladır taksi dksı görünen otobüsü bekliyoruz taksi reddetti istanbulun ulaşım sıkıntısından gerçekten yıldım taksi ibbşikayet',
 'olimpiyat cadde üzeri pendik buraya araç park etmek yasak arkadaslar usulsüz şekilde park ediyorotobüs ler araç lar zaman zaman zorlanıyorgeçmekte ibbşikayet emniyet trafikşube',
 'parklarda kontrol bakımları aksatarak canlarla oynuyorsunuz kadikoybelediyesi kadikoysikayet',
 'parklarda kontrol bakımları aksatarak canlarla oynuyorsunuz kadikoybelediyesi kadikoysikayet',
 'bugün bir taksici terörüne tanık oldumtaksici bagdat cadkarşıya geçmekte bir çifti kalsın eziyorduadam dikkat etmesini söyleyince küfrettikavga çıkarttı adamın gözüne sprey gaz sıkıp kaçtıbu taksici tehlikelitjc ibbşikayet taksiciterörü']

In [ ]:
model = BERTopic(language="turkish")

In [ ]:
topics, probs = model.fit_transform(docs)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
model.get_topic_freq()

,Topic,Count
0,-1,3414
1,0,544
2,1,360
3,2,286
4,3,237
...,...,...
95,94,11
96,95,10
97,96,10
98,97,10


In [ ]:
model.get_topic(2)

[('metro', 0.06737428740033828),
 ('metrobüs', 0.02092317440756054),
 ('metrosu', 0.01669639277082499),
 ('metroyu', 0.015504848598950174),
 ('metronun', 0.013680748763779563),
 ('dudullu', 0.013453814986712796),
 ('metroya', 0.012396281047475656),
 ('durağı', 0.01214283110605454),
 ('fulya', 0.012138135568201836),
 ('bostancı', 0.011740894554428024)]

In [ ]:
model.get_topic(4)

[('otobüs', 0.0714009973228333),
 ('otobüsü', 0.04075103558797718),
 ('otobüsler', 0.028688532266003493),
 ('otobüsleri', 0.023702969592990057),
 ('iett', 0.02055689219713246),
 ('otobüse', 0.015045537161375008),
 ('saat', 0.014909171147908611),
 ('durakta', 0.012888511902611088),
 ('geç', 0.012683305549045388),
 ('st', 0.012649172531846544)]

In [ ]:
model.get_topic(5)

[('doğalgaz', 0.05196924500195601),
 ('indirim', 0.03526776024485126),
 ('gaz', 0.0328749322371484),
 ('indirimi', 0.02662509536921511),
 ('doğal', 0.023808887660557363),
 ('zam', 0.02172800740954839),
 ('gazı', 0.019090934830791946),
 ('benzin', 0.018259872548932486),
 ('elektrik', 0.01771569300576801),
 ('igdaş', 0.01705219241131573)]

In [ ]:
model.get_topic(10)

[('taksi', 0.09021796212374494),
 ('taksici', 0.04522782586104116),
 ('taksiler', 0.028102619332341514),
 ('korsan', 0.01718476218695371),
 ('yolcu', 0.01668632062681343),
 ('taksicilerin', 0.016394127961035572),
 ('taksicilik', 0.016394127961035572),
 ('taksilerin', 0.01574849546458103),
 ('taksinin', 0.01574849546458103),
 ('taksiciler', 0.014776221095055027)]

BerTopic Görselleştirme

In [ ]:
model.visualize_topics()